### 9.0 Overview

q-sql vs SQL

The first important difference is that a q table has ordered rows and columns. This is particularly useful when dealing with the situation where records arrive in time order. Appending them to a table ensures that they enter – and stay – in order. Subsequent select operations always retrieve the records in order without any need for sorting.


A second difference is that a q table is stored physically as a collection of column lists. This means that operations on column data are vector operations. Moreover, for simple column lists, atomic, aggregate and uniform functions applied to columns are especially simple and fast since they reduce to direct memory addressing.


A third difference is that q-sql provides upsert semantics. Recall that upsert semantics on a dictionary mean that when a key-value pair is applied with , and the key is present, the value is updated; otherwise the pair is inserted. In the context of tables and keyed tables, which are both dictionaries, this has far-reaching consequences for many common operations, including joins. Upsert semantics permeate q-sql.

#### 9.1 Inserting Records

The upsert function is superior to insert and is to be preferred. We include insert for nostalgia only.

#### 9.1.0 Append Using Amend

Since a table is (logically) a list of records, it is possible to append records in place using ,:. 

In [1]:
t:([] name:`symbol$(); iq:`int$())
t,:`name`iq!(`Beeblebrox; 42)

Amend can also be used with a row of naked field values provided the fields align exactly with the target columns. ( no column names)

In [2]:
t,:(`Prefect; 126)
t

name       iq 
--------------
Beeblebrox 42 
Prefect    126


**Append into keyed table**

In [3]:
kt:([eid:`long$()] name:`symbol$(); iq:`long$())  / keyed table

In [4]:
kt,:(enlist (enlist `eid)!enlist 1001)!enlist `name`iq!(`Beeblebrox; 42)  / full form - unreadable

In [5]:
kt,:(1002; `Dent; 98) / naked values are much cleaner
kt

eid | name       iq
----| -------------
1001| Beeblebrox 42
1002| Dent       98


Amend has upsert semantics on keyed tables, so repeated operation on the same key will retain only the last values.

In [6]:
kt,:(1002; `Dent; 99)
kt

eid | name       iq
----| -------------
1001| Beeblebrox 42
1002| Dent       99


#### 9.1.1 Basic insert

sql-like insert

In [7]:
t:([] name:`Dent`Beeblebrox`Prefect; iq:42 98 126)
t
`t insert (`name`iq)!(`Slartibartfast; 134) / full form, returns row numbers
t
`t insert (`Marvin; 150)    / naked values
t

name       iq 
--------------
Dent       42 
Beeblebrox 98 
Prefect    126


,3


name           iq 
------------------
Dent           42 
Beeblebrox     98 
Prefect        126
Slartibartfast 134


,4


name           iq 
------------------
Dent           42 
Beeblebrox     98 
Prefect        126
Slartibartfast 134
Marvin         150


Repeatedly applying insert to a table with the same argument results in duplicate records.

In [8]:
t:3#t  / taking only first 3 records, removing last 2

In [9]:
`t insert (`name`iq!(`Slartibartfast; 134); (`name`iq!(`Marvin; 200))) / multiple records in one insert
t
t:3#t
t
`t insert ([] name:`Slartibartfast`Marvin; iq:134 200)
t

3 4


name           iq 
------------------
Dent           42 
Beeblebrox     98 
Prefect        126
Slartibartfast 134
Marvin         200


name       iq 
--------------
Dent       42 
Beeblebrox 98 
Prefect    126


3 4


name           iq 
------------------
Dent           42 
Beeblebrox     98 
Prefect        126
Slartibartfast 134
Marvin         200


In [10]:
insert[`t; (`Slartibartfast; 134)] / prefix form
insert[`t;] (`Slartibartfast; 134) / prefix form, table name projected

,5


,6


#### 9.1.2 Bulk Columnar Insert

In [11]:
t:([] name:`Dent`Beeblebrox; iq:98 42)
`t insert (`Prefect; 126)  / naked single-row insert
t

,2


name       iq 
--------------
Dent       98 
Beeblebrox 42 
Prefect    126


How to insert multiple naked records in one go

In [12]:
/ `t insert ((`Prefect; 126);(`Slartibartfast; 134))  / naked multiple-row insert NOT WORKING

To bulk insert naked field values, you provide a list of columns not a list of rows.

In [13]:
`t insert (`Prefect`Slartibartfast; 126 134)
t

3 4


name           iq 
------------------
Dent           98 
Beeblebrox     42 
Prefect        126
Prefect        126
Slartibartfast 134


#### 9.1.3 Insert into Empty Tables

Inserting into a table that has been initialized with empty lists of general type causes the result table to take the type of the first record inserted. In particular, an atomic field in the inserted record results in a simple column with its type.

In [14]:
t:([] name:(); iq:())
`t insert (`Dent;98)
meta t

,0


c   | t f a
----| -----
name| s    
iq  | j    


It is good practice to type all columns in an empty table. This will ensure that incorrect types are rejected and correct ones accepted.

It is also possible to insert a list of conforming records (i.e., a table) into a table that does not exist. This is the same as assigning the table to a variable of the specified name.

In [15]:
`tnew insert enlist `c1`c2!(`a; 10)
tnew

,0


c1 c2
-----
a  10


#### 9.1.4 Insert and Foreign Keys

When inserting data into a table that has foreign key(s), the values destined for the foreign key column(s) are checked to ensure that they appear in the primary key column(s) pointed to by the foreign key(s). This is referential integrity (well, half of it).

In [16]:
kt:([eid:1001 1002 1003] name:`Dent`Beeblebrox`Prefect; iq:98 42 126)
tdetails:([] eid:`kt$1003 1002 1001 1002 1001; sc:126 36 92 39 98)

In [17]:
`tdetails insert (1002;42)

,5


In [18]:
/ `tdetails insert (1062;82)   FK violation

#### 9.1.5 Insert into Keyed Tables

You can use insert to append data to a keyed table, but this probably does not have the desired semantics. Specifically, you can insert into a keyed table only if the key value is not already in the table. For this and other reasons, upsert should normally be preferred over insert.


In [19]:
kt:([eid:1001 1002] name:`Dent`Beeblebrox; iq:98 42)

In [20]:
`kt insert (1005; `Marvin; 200)

,2


In [21]:
/ `kt insert (1005; `Marvin; 200)  - ERROR: duplicate

### 9.2 Upsert

The upsert template is like insert, only better. 

#### 9.2.1 Upsert Replacing insert

In [22]:
t:([] name:`Dent`Beeblebrox`Prefect; iq:42 98 126);
`t upsert (`name`iq)!(`Slartibartfast; 134)
t
`t upsert (`Marvin; 150)
t
`t upsert ([] name:`Slartibartfast`Marvin; iq:134 200)
t
t:3#t
t
upsert[`t; (`Slartibartfast; 134)]
t

`t


name           iq 
------------------
Dent           42 
Beeblebrox     98 
Prefect        126
Slartibartfast 134


`t


name           iq 
------------------
Dent           42 
Beeblebrox     98 
Prefect        126
Slartibartfast 134
Marvin         150


`t


name           iq 
------------------
Dent           42 
Beeblebrox     98 
Prefect        126
Slartibartfast 134
Marvin         150
Slartibartfast 134
Marvin         200


name       iq 
--------------
Dent       42 
Beeblebrox 98 
Prefect    126


`t


name           iq 
------------------
Dent           42 
Beeblebrox     98 
Prefect        126
Slartibartfast 134


To bulk upsert naked field values, use rows instead of columns. ( for insert we used columns instead of rows)

In [23]:
t upsert ((`Prefect; 126); (`Marvin; 200))

name           iq 
------------------
Dent           42 
Beeblebrox     98 
Prefect        126
Slartibartfast 134
Prefect        126
Marvin         200


#### 9.2.2 Upsert by Name

A limitation of insert is that it uses pass-by-name, so it can only operate against global tables. In contrast, upsert supports both pass-by-name and pass-by-value. Thus it can be used with anonymous or local tables.

In [24]:
([] c1:`a`b; c2:10 20) upsert (`c; 30)

c1 c2
-----
a  10
b  20
c  30


In [25]:
f:{t:([] c1:`a`b; c2:10 20); t upsert x}
f (`c; 30)

c1 c2
-----
a  10
b  20
c  30


#### 9.2.3 Upsert on Keyed Tables

We have seen that insert has undesirable semantics on keyed tables – i.e., it rejects “duplicate” keys. What we really want is, well, upsert semantics.
This is the second reason to use upsert instead of insert.

In [26]:
kt:([eid:1001 1002] name:`Dent`Beeblebrox; iq:98 42)
kt
`kt upsert (1002; `Beeblebrox; 42)
kt
`kt upsert (1002; `Beeblebrox; 43)
kt

eid | name       iq
----| -------------
1001| Dent       98
1002| Beeblebrox 42


`kt


eid | name       iq
----| -------------
1001| Dent       98
1002| Beeblebrox 42


`kt


eid | name       iq
----| -------------
1001| Dent       98
1002| Beeblebrox 43


#### 9.2.4 Upsert on Persisted Tables

You can use upsert to append records to serialized and splayed tables. Simply pass the handle of the file or splayed directory as the name of the table. This is the final strike against insert, since it cannot do this.

**We serialize a table and then append a row to it.**

In [27]:
`:C:/Dev/kdb_q/notebooks/q_for_mortals/tser set ([] c1:`a`b; c2:1.1 2.2)

`:C:/Dev/kdb_q/notebooks/q_for_mortals/tser


In [28]:
`:C:/Dev/kdb_q/notebooks/q_for_mortals/tser upsert (`c; 3.3)


`:C:/Dev/kdb_q/notebooks/q_for_mortals/tser


In [29]:
get `:C:/Dev/kdb_q/notebooks/q_for_mortals/tser

c1 c2 
------
a  1.1
b  2.2
c  3.3


Upserting to a serialized table reads the entire table into memory, updates it and writes out the result.

**Splayed**

to be added..

### 9.3 The select Template

The template is converted by the interpreter into a functional form and is applied against the table to produce a result table.
The result of select is always a table.

#### 9.3.1 Syntax

The select template has the following form, where elements enclosed in matching angle brackets <...> are optional.

select <ps> <by pb> from texp <where pw>
    
Each phrase in the select template is a comma-separated list of subphrases. A subphrase is an arbitrary q expression (presumably) involving columns of texp or columns of another table accessed via foreign key. The evaluation of subphrases within a phrase is sequenced left-to-right by the commas, but each subphrase expression is evaluated right-to-left, like any q expression.

#### 9.3.2 The select Phrase

In [30]:
t:([] c1:`a`b`c; c2:10 20 30; c3:1.1 2.2 3.3)

- The colon used to specify a name is not assignment; it is simply part of the syntax of the template.

- Just as with the use of colon in a variable assignment, the column name is part of the syntactic sugar. It is not a symbol and cannot be parameterized. Use functional form if you need this – see §9.12.

- Unlike in SQL, columns in the select phrase do not actually exist until the final result table is returned. Thus a computed column cannot be used in other column expressions.

A virtual column i represents the offset of each record in the table – i.e., i is the row number. It is implicitly available in the select phrase.

In [31]:
select i, c1 from t

x c1
----
0 a 
1 b 
2 c 


In [32]:
select distinct from ([] c1:`a`b`a; c2:10 20 10) / removed duplicates

c1 c2
-----
a  10
b  20


#### 9.3.2.4 select[]

You can return the first or last n records in a select by using function parameter syntax after select. A positive integer parameter returns the first records, a negative parameter the last.

In [ ]:
\l C:/Dev/kdb_q/notebooks/q_for_mortals/sp.q

In [34]:
select[2] from s where city<>`athens
2#select from s where city<>`athens

s | name  status city  
--| -------------------
s1| smith 20     london
s2| jones 10     paris 


s | name  status city  
--| -------------------
s1| smith 20     london
s2| jones 10     paris 


In [35]:
select[-1] from s where city<>`athens
-1#select from s where city<>`athens

s | name  status city  
--| -------------------
s4| clark 20     london


s | name  status city  
--| -------------------
s4| clark 20     london


The difference is that the # construct requires computing the entire result set and then keeping only the desired rows, whereas select[n] only extracts the desired number of rows. The latter will be faster and consume less memory for large tables.

This syntax is extended to select[n m] where m is the starting row number and n is the number of rows.

In [36]:
select[2 2] from s where city<>`athens

s | name  status city  
--| -------------------
s3| blake 30     paris 
s4| clark 20     london


One final extension of the syntax specifies a sorting criterion inside the brackets. For ascending sort, place < before a column name and for descending sort use `>.

In [37]:
select[>name] from s where city<>`athens

s | name  status city  
--| -------------------
s1| smith 20     london
s2| jones 10     paris 
s4| clark 20     london
s3| blake 30     paris 


You can combine the two syntax extensions by separating them with a semicolon.

In [38]:
select[2; >name] from s where city<>`athens

s | name  status city  
--| -------------------
s1| smith 20     london
s2| jones 10     paris 


#### 9.3.3 Filtering with where

Recall that a table is logically a list of records.

In [39]:
t:([] c1:`a`b`c; c2:10 20 30; c3:1.1 2.2 3.3)

In [40]:
t where t[`c2]>15
select from t where c2>15

c1 c2 c3 
---------
b  20 2.2
c  30 3.3


c1 c2 c3 
---------
b  20 2.2
c  30 3.3


#### 9.3.3.2 The Virtual Column i in where ( pagination )

The virtual column i is useful for **paginating** a table. Use within, which returns a boolean indicating whether the left operand is in the closed interval specified in the right operand, to determine the bounds of the page.

In [41]:
tbig:100#t
tbig

c1 c2 c3 
---------
a  10 1.1
b  20 2.2
c  30 3.3
a  10 1.1
b  20 2.2
c  30 3.3
a  10 1.1
b  20 2.2
c  30 3.3
a  10 1.1
b  20 2.2
c  30 3.3
a  10 1.1
b  20 2.2
c  30 3.3
a  10 1.1
b  20 2.2
c  30 3.3
a  10 1.1
b  20 2.2
..


In [42]:
select from tbig where i within 50 99

c1 c2 c3 
---------
c  30 3.3
a  10 1.1
b  20 2.2
c  30 3.3
a  10 1.1
b  20 2.2
c  30 3.3
a  10 1.1
b  20 2.2
c  30 3.3
a  10 1.1
b  20 2.2
c  30 3.3
a  10 1.1
b  20 2.2
c  30 3.3
a  10 1.1
b  20 2.2
c  30 3.3
a  10 1.1
..


#### 9.3.3.3 Multiple where Subphrases

In [43]:
r1:select from t where c2>15,c3<3.0
r2:select from t where (c2>15)&c3<3.0 / Note that the parentheses are necessary in the second query.
r1~r2

1b


However, since the where subphrases are sequenced from left-to-right, their order affects the actual processing. As each subphrase is applied, only the records it passes are tested in the next subphrase. The net effect is a progressively narrowed sublist of rows to consider. There is often an optimal order that significantly narrows in the first one or few subphrases, which in turn reduces the amount of processing.

Tip

**Place the most limiting where subphrase first, followed by others in decreasing strictness.**

**Comparing string column**

In [44]:
t:([] f:1.1 2.2 3.3; s:("abc";enlist "d";"ef"))

In [45]:
"ef"~"ef"

1b


In [46]:
select from t where s~\:"ef" / each-left works
select from t where s like "ef" / fastest

f   s   
--------
3.3 "ef"


f   s   
--------
3.3 "ef"


#### 9.3.3.5 fby in where

In SQL you would use HAVING, but q is having none of that. Instead use fby in the where phrase. Since it returns the value of the aggregate across each group, you simply compare the target column to the fby result to get a boolean vector with 1b at precisely the records whose fields match the aggregate on the group.

Used in a where phrase, fby takes the form

(fagg;exprcol) fby c

The left operand is a two-item list comprising an aggregate function fagg and a column expression exprcol on which the function will be applied. The right operand c is the column to be grouped.

In [47]:
select from p where weight=(max;weight) fby city

p | name  color weight city  
--| -------------------------
p2| bolt  green 17     paris 
p3| screw blue  17     rome  
p6| cog   red   19     london


In [48]:
select from p where weight=(max;weight) fby city,color=`blue

p | name  color weight city
--| -----------------------
p3| screw blue  17     rome


grouping on multiple columns -  encapsulate them in an anonymous table in the right operand of fby.

In [49]:
t:([]sym:`IBM`IBM`MSFT`IBM`MSFT;
    ex:`N`O`N`N`N;
    time:12:10:00 12:30:00 12:45:00 12:50:00 13:30:00;
    price:82.1 81.95 23.45 82.05 23.40)
select from t where price=(max;price) fby ([]sym;ex)

sym  ex time     price
----------------------
IBM  N  12:10:00 82.1 
IBM  O  12:30:00 81.95
MSFT N  12:45:00 23.45


#### 9.3.4 Grouping and Aggregation

In contrast to SQL, where grouping and aggregation are performed together, in q-sql they are independent. 

#### 9.3.4.1 Aggregation without Grouping

When an aggregate function is applied against a column of simple type in the select phrase, the result is an atom. If all columns in the select phrase are computed with aggregation and there is no grouping, the result will be a table with a single row – e.g., a summary or rollup. While q has many built-in aggregates, you can also define and use your own.

In [50]:
select total:sum qty, mean:avg qty from sp / no grouping

total mean    
--------------
3100  258.3333


In [51]:
select total:sum qty, qty from sp / no grouping

total qty                                            
-----------------------------------------------------
3100  300 200 400 200 100 100 300 400 200 200 300 400


#### 9.3.4.2 Grouping without Aggregation

- The by phrase groups rows having common values in specified column(s), much like GROUP BY in SQL. 

- The result of a query including a by phrase is a keyed table whose key column(s) are those in the by phrase. This is well-defined because the grouping along like values ensures uniqueness of the keys.

- A query that groups without aggregation results in nested columns. One way to think of this is that each group of values is folded into a single field in the result.

In [52]:
t:([] c1:`a`b`a`b`c; c2:10 20 30 40 50)

In [53]:
t[`c2] group t[`c1]
select c2 by c1 from t

a| 10 30
b| 20 40
c| ,50


c1| c2   
--| -----
a | 10 30
b | 20 40
c | ,50  


Observe that **ungroup** can be used to reverse the nested result of grouping without aggregation. It is not quite an inverse since it returns the original records ordered on the by column(s).

In [54]:
ungroup  select c2 by c1 from t

c1 c2
-----
a  10
a  30
b  20
b  40
c  50


There are use cases that group on specified column(s) and want all the remaining columns to be nested in the result. 

In [55]:
p1:`p xgroup sp
p1
p2:select s,qty by p from sp
p2
p1~p2

p | s               qty            
--| -------------------------------
p1| `s$`s1`s2       300 300        
p2| `s$`s1`s2`s3`s4 200 400 200 200
p3| `s$,`s1         ,400           
p4| `s$`s1`s4       200 300        
p5| `s$`s4`s4       100 400        
p6| `s$,`s1         ,100           


p | s               qty            
--| -------------------------------
p1| `s$`s1`s2       300 300        
p2| `s$`s1`s2`s3`s4 200 400 200 200
p3| `s$,`s1         ,400           
p4| `s$`s1`s4       200 300        
p5| `s$`s4`s4       100 400        
p6| `s$,`s1         ,100           


1b


In [56]:
select s,qty by p from sp

p | s               qty            
--| -------------------------------
p1| `s$`s1`s2       300 300        
p2| `s$`s1`s2`s3`s4 200 400 200 200
p3| `s$,`s1         ,400           
p4| `s$`s1`s4       200 300        
p5| `s$`s4`s4       100 400        
p6| `s$,`s1         ,100           


#### 9.3.4.3 Grouping with Aggregation

In [57]:
t:([] desk:`a`b`a`b`a`b; acct:`1`2`3`4`1`4; pnl:1.1 -2.2 3.3 4.4 5.5 -.5)
select ct:count desk, sum pnl by desk,acct from t


desk acct| ct pnl 
---------| -------
a    1   | 2  6.6 
a    3   | 1  3.3 
b    2   | 1  -2.2
b    4   | 2  3.9 


In contrast to SQL, every column in the by phrase is automatically included in the key column(s) of the result and should not be duplicated in the select phrase.

A by subphrase can be a q expression, meaning that you can group on computed columns. This is very powerful and is not present in SQL. Following is a useful example that averages the observations of our time series in 100 millisecond buckets.

In [58]:
t:([] c1:00:00:00.000+til 1000000;c2:1000000?`a`b;c3:1000000?100.)
select avg c3 by 100 xbar c1, c2 from t

c1           c2| c3      
---------------| --------
00:00:00.000 a | 55.26494
00:00:00.000 b | 41.81758
00:00:00.100 a | 48.88826
00:00:00.100 b | 46.10946
00:00:00.200 a | 53.72272
00:00:00.200 b | 51.42873
00:00:00.300 a | 54.53996
00:00:00.300 b | 49.50472
00:00:00.400 a | 51.95785
00:00:00.400 b | 53.63795
00:00:00.500 a | 55.24681
00:00:00.500 b | 58.08108
00:00:00.600 a | 40.27698
00:00:00.600 b | 53.36846
00:00:00.700 a | 46.88679
00:00:00.700 b | 50.91821
00:00:00.800 a | 54.62589
00:00:00.800 b | 58.76686
00:00:00.900 a | 53.28107
00:00:00.900 b | 48.06001
..


In [59]:
select  by p from sp / equivalent of applying last to all columns

p | s  qty
--| ------
p1| s2 300
p2| s4 200
p3| s1 400
p4| s4 300
p5| s4 400
p6| s1 100


### 9.4 The exec Template

The syntax of the exec template is identical to that of select.

exec <ps> <by pb> from texp <where pw>

Whereas select always returns a table, the result type of exec depends on the number of columns in its select phrase. One column yields a list; more than one column yields a dictionary.

When more than one column is specified the select phrase, the result is a dictionary mapping column names to the column lists produced. 

In [60]:
t:([] name:`a`b`c`d`e; state:`NY`FL`OH`NY`HI)

In [61]:
/ select name, distinct state from t  / error

In [62]:
exec name, distinct state from t

name | `a`b`c`d`e
state| `NY`FL`OH`HI


In [63]:
select name from t
exec name from  t / retruns list

name
----
a   
b   
c   
d   
e   


`a`b`c`d`e


### 9.5 The update Template

#### 9.5.1 Basic update

The update template has identical syntax to select.

update <pu> <by pb> from texp <where pw>

In [64]:
t:([] c1:`a`b`c; c2:10 20 30)
t
update c1:`x`y`z from `t
t
update c3:`x`y`z from `t
t

c1 c2
-----
a  10
b  20
c  30


`t


c1 c2
-----
x  10
y  20
z  30


`t


c1 c2 c3
--------
x  10 x 
y  20 y 
z  30 z 


#### 9.5.2 update-by

When the by phrase is present, the update operation is performed along groups. This is most useful with aggregate and uniform functions. For an aggregate function, the entire group gets the value of the aggregation on the group.

In [65]:
update avg weight by city from p

p | name  color weight city  
--| -------------------------
p1| nut   red   15     london
p2| bolt  green 14.5   paris 
p3| screw blue  17     rome  
p4| screw red   15     london
p5| cam   blue  14.5   paris 
p6| cog   red   15     london


A uniform function is applied along the group in place. This can be used to compute cumulative volume of orders, or rank, for example.

In [66]:
update cumqty:sums qty by s from sp

s  p  qty cumqty
----------------
s1 p1 300 300   
s1 p2 200 500   
s1 p3 400 900   
s1 p4 200 1100  
s4 p5 100 100   
s1 p6 100 1200  
s2 p1 300 300   
s2 p2 400 700   
s3 p2 200 200   
s4 p2 200 300   
s4 p4 300 600   
s4 p5 400 1000  


### 9.6 The delete Template

**delete**, allows either rows or columns to be deleted. Its syntax is a simplified form of select, with the restriction that **either pcols or pw can be present but not both**.

**delete pcols from texp where pw**

In [67]:
t:([] c1:`a`b`c; c2:10 20 30)
delete c1 from t
delete from t where c2>15

c2
--
10
20
30


c1 c2
-----
a  10


**Tip**. When you want to select all but a few columns, it is easier to delete the ones you don’t want than list all the ones you do.

In [68]:
t:([] c1:1 2; c2:`a`b; c3:1.1 2.2; c4:2015.01.01 2015.01.02)
(select c1, c2, c4 from t)~delete c3 from t

1b


### 9.7 Sorting

Recall that tables and keyed tables comprise lists of records and therefore have an inherent order. A table or keyed table can be reordered by sorting on any column(s). In contrast to SQL, there is no equivalent to ORDER BY in the select template. Instead, built-in functions that sort tables are applied after select.

#### 9.7.1 xasc

In [69]:
t:([] c1:`a`b`c`a; c2:20 10 40 30)
`c1`c2 xasc t / sort copy
t
`c1`c2 xasc `t / sort in-place
t

c1 c2
-----
a  20
a  30
b  10
c  40


c1 c2
-----
a  20
b  10
c  40
a  30


`t


c1 c2
-----
a  20
a  30
b  10
c  40


The dyadic **xdesc** behaves exactly as xasc, except that the sort is performed in descending order.

#### 9.7.3 Mixed Sort

We point out that xasc and xdesc are stable sorts, meaning that the order of two records having the same sort key value is preserved in the result. This makes it possible to compose ascending and descending sort to obtain mixed sorts. For example, to sort c2 descending within c1 ascending in t above,

In [70]:
`c1 xasc `c2 xdesc t

c1 c2
-----
a  30
a  20
b  10
c  40


### 9.8 Renaming and Rearranging Columns

In [71]:
t:([] c1:`a`b`c; c2:10 20 30; c3:1.1 2.2 3.3)

#### 9.8.1 xcol

In [72]:
`new1`new2 xcol t   / renames left-most columns
t

new1 new2 c3 
-------------
a    10   1.1
b    20   2.2
c    30   3.3


c1 c2 c3 
---------
a  10 1.1
b  20 2.2
c  30 3.3


You can use constructs such as the following to rename isolated columns if your table has many columns and the targeted columns area not left-most.

In [73]:
@[cols[t]; where cols[t]=`c2; :; `new2] xcol t

c1 new2 c3 
-----------
a  10   1.1
b  20   2.2
c  30   3.3


In [74]:
@[cols[t]; where cols[t] in `c1`c3; :; `new1`new3] xcol t

new1 c2 new3
------------
a    10 1.1 
b    20 2.2 
c    30 3.3 


#### 9.8.2 xcols

rearranges columns. Those which are left operand become first, others order is unchanged.

In [75]:
t


c1 c2 c3 
---------
a  10 1.1
b  20 2.2
c  30 3.3


In [76]:
`c3 xcols t

c3  c1 c2
---------
1.1 a  10
2.2 b  20
3.3 c  30


In [77]:
`c3`c2 xcols t

c3  c2 c1
---------
1.1 10 a 
2.2 20 b 
3.3 30 c 


### 9.9 Joins

Joins can also be classified by how key matching is determined. 
- Most joins are **equijoins**, meaning that the keys must be equal. 
- In q there are also non-equijoins, called **as of joins**, in which a key is tested for less-than-or-equal against keys in another table.

#### 9.9.1 Implicit Join

Given a primary key table m with key column(s) k and a table d with a foreign key linking to k.
A foreign key in q is accomplished with an enumeration over the key column(s) of a keyed table. 

**A left join is implicit in the following query on the detail table.**

select cold, k.colm from d

In [78]:
select sname:s.name, pname:p.name, qty from sp

sname pname qty
---------------
smith nut   300
smith bolt  200
smith screw 400
smith screw 200
clark cam   100
smith cog   100
jones nut   300
jones bolt  400
blake bolt  200
clark bolt  200
clark screw 300
clark cam   400


Implicit joins extend to the situation in which the targeted keyed table itself has a foreign key to another keyed table.

In [85]:
emaster:([eid:1001 1002 1003 1004 1005] currency:`gbp`eur`eur`gbp`eur)
update eid:`emaster$1001 1002 1005 1004 1003 from `s
select s.name, qty, s.eid.currency from sp

`s


name  qty currency
------------------
smith 300 gbp     
smith 200 gbp     
smith 400 gbp     
smith 200 gbp     
clark 100 gbp     
smith 100 gbp     
jones 300 eur     
jones 400 eur     
blake 200 eur     
clark 200 gbp     
clark 300 gbp     
clark 400 gbp     


#### 9.9.2 Ad hoc Left Join (lj)

- The right operand is a **keyed table (target)** and 
- the left operand is a table or keyed table (source) having either a foreign key to target or column(s) that match the key column(s) of target in name and type. 

- Observe that when the source table has a foreign key, an ad-hoc left join is equivalent to listing all columns from both tables in an implicit join.

- When the tables source and target have duplicate non-key columns, the operation has upsert semantics.
That is, the values in the right operand (target) columns prevail over those in the left operand (source). This is different from SQL where the result contains both columns with suffixes to ensure unique names.

In [153]:
t:([] k:1 2 3 4; c:10 20 30 40; d:210 220 230 240 )
kt:([k:2 3 4 5 3]; v:200 300 400 500 600; d:21 22 23 24 25) / match will happen only on first record where k=3. Second will be ignored
t lj kt

k c  d   v  
------------
1 10 210    
2 20 21  200
3 30 22  300
4 40 23  400


In [154]:
(select from (t lj kt) where null v) uj ungroup t ij `k xgroup kt

k c  d   v  
------------
1 10 210    
2 20 21  200
3 30 22  300
3 30 25  600
4 40 23  400


#### 9.9.3 Column Lookup

You can perform a column lookup against a keyed table within a query without using a join. The insight is that a keyed table is a dictionary whose key list comprises its key records, so it will perform the lookup provided we put the column in an anonymous table conforming to those key records – see §8.4.5. Here we demonstrate the case where the lookup column names do not natively match the key columns, so we rename columns to match in the anonymous table.

In [80]:
t:([] k:1 2 3 4; c:10 20 30 40)
kt:([k1:2 3 4 5]; v:2.2 3.3 4.4 5.5)


In [81]:
kt[([] k1:t[`k]); `v]  / returns column v of kt for keys in t['k]

0n 2.2 3.3 4.4


In [82]:
select c, v:kt[([] k1:k); `v] from t

c  v  
------
10    
20 2.2
30 3.3
40 4.4


Especially for a single column, this is simpler (and more impressive to your colleagues) than the equivalent join.

In [83]:
select c,v from t lj `k xkey select k:k1,v from kt

c  v  
------
10    
20 2.2
30 3.3
40 4.4


#### 9.9.4 Ad Hoc Inner Join (ij)

The dyadic inner-join operator ij performs an inner join between two tables that could have a foreign key relationship.

As with lj, the right operand is a keyed table (target) and the left operand is a table or keyed table (source) having column(s) that are either foreign key(s) over target or exactly match the key column(s) of target in name and type. The matching is done via the foreign key or by common column name(s) between source and the key column(s) of target if there is no foreign-key relationship. The result contains the columns from source and target joined along common keys.

As with lj, upsert semantics holds for duplicate columns.

Works for keyed tables as well.


In [105]:
t:([] k:1 2 3 4; c:10 20 30 40)
kt:([k:2 3 4 5 2]; v:2.2 3.3 4.4 5.5 6.6)
t ij kt   / returns only first match from lookup table

k c  v  
--------
2 20 2.2
3 30 3.3
4 40 4.4


#### 9.9.5 Equijoin ej

The triadic equijoin operator ej corresponds to a SQL inner join between tables in the second and third parameters along specified column names in the first parameter. The right operand does not have to be a keyed table. Unlike ij, all matching records in the right table appear in the result. As with any join, upsert semantics holds on duplicate columns.

In [86]:
t1:([] k:1 2 3 4; c:10 20 30 40)
t2:([] k:2 2 3 4 5; c:200 222 300 400 500; v:2.2 22.22 3.3 4.4 5.5)
t1 ij `k xkey t2    / inner join - only 1 matching record from right table

k c   v  
---------
2 200 2.2
3 300 3.3
4 400 4.4


In [87]:
ej[`k;t1;t2] / equijoin - all matching records from right table

k c   v    
-----------
2 200 2.2  
2 222 22.22
3 300 3.3  
4 400 4.4  


#### 9.9.6 Plus Join (pj)

Plus join pj is a left join that replaces upsert semantics for duplicate column names with addition. This is useful when you have two tables with identical schemas having all non-key columns numeric. For example, you have an organizational hierarchy and you want to roll up numeric results.

The operands of pj are the same as for lj with the additional requirement that all non-key columns are numeric. The semantics are that duplicate columns are added along matching keys and missing or null values are treated as zero.

In [89]:
t:([] k:`a`b`c; a:100 200 300; b:10. 20. 30.; c:1 2 3)
kt:([k:`a`b] a:10 20; b:1.1 2.2)
t pj kt

k a   b    c
------------
a 110 11.1 1
b 220 22.2 2
c 300 30   3


#### 9.9.7 Union Join

The equivalent of an ordinary SQL union on tables with matching schemas is simply ,. Indeed, it joins two lists of compatible records.

In [115]:
t1:([] c1:`a`b; c2:1 2)
t2:([] c1:`c`d; c2:3 4)
t1,t2

c1 c2
-----
a  1 
b  2 
c  3 
d  4 


Union join deals with different schemas - adds columns.
Upsert for keyed tables

In [116]:
t1:([] c1:`a`b`c; c2: 10 20 30)
t2:([] c1:`x`y; c3:8.8 9.9)
t1 uj t2

c1 c2 c3 
---------
a  10    
b  20    
c  30    
x     8.8
y     9.9


In [117]:
t3:([] c1:`e`f`g; c2:50 60 70; c3:5.5 6.6 7.7)
(uj/) (t1; t2; t3)

c1 c2 c3 
---------
a  10    
b  20    
c  30    
x     8.8
y     9.9
e  50 5.5
f  60 6.6
g  70 7.7


#### 9.9.8 As-of Joins

As-of joins are so-named because they most often join tables along time columns to obtain a value in one table that is current as of a time in another table. As-of joins are non-equijoins that match on less-than-or-equal. They are not restricted to time values.

The fundamental as-of join is the triadic function aj. It joins tables along common columns using most recent values. The syntax of aj is,

aj[`c1...`cn;t1;t2]

where `c1...`cn is a simple list of symbol column names common to t1 and t2, which are the tables to be joined. There is no requirement for any of the join columns to be keys but the join will be faster on keys. The columns of both tables are brought into the result.

The semantics of aj are as follows. The match on all specified columns except the last is by equals. Assuming the records are sequenced by cn in both tables, for a given cn value in t1, the match picks the greatest cn in t2 less than or equal to the given value in t1. Specifically, if the cn columns are sequenced temporal values, for each cn value in t1 the match picks the t2 row whose cn value is in effect “as of” the time in t1.

The canonical example for aj is matching trades with quotes. To know if a trade represents best execution, you want to compare the trade price to the current quote – i.e., the most recent quote up to and including the time of the trade. When matching the trades and quotes, you clearly want an equi-join on the date and symbol and a non-equijoin on the time.

In [118]:
show t:([] ti:10:01:01 10:01:03 10:01:04;sym:`msft`ibm`ge;qty:100 200 150)
show q:([] ti:10:01:00 10:01:01 10:01:01 10:01:02;sym:`ibm`msft`msft`ibm;px:100 99 101 98)

ti       sym  qty
-----------------
10:01:01 msft 100
10:01:03 ibm  200
10:01:04 ge   150
ti       sym  px 
-----------------
10:01:00 ibm  100
10:01:01 msft 99 
10:01:01 msft 101
10:01:02 ibm  98 


In [119]:
aj[`sym`ti;t;q]

ti       sym  qty px 
---------------------
10:01:01 msft 100 101
10:01:03 ibm  200 98 
10:01:04 ge   150    


If you want the time of the matching quote in the result instead of the time of the trade, use **aj0**

In [120]:
aj0[`sym`ti;t;q]

ti       sym  qty px 
---------------------
10:01:01 msft 100 101
10:01:02 ibm  200 98 
10:01:04 ge   150    


The simpler function **asof** performs the same match as aj but with a table against a single record. The result picks out the remaining columns in the matched row of the table.

In [122]:
t:([] ti:10:01:01 10:01:03 10:01:04; sym:`msft`ibm`ge; qty:100 200 150; px:45 160 55)
t asof `sym`ti!(`ibm;10:01:03)

qty| 200
px | 160


A list of such dictionary records conforms – i.e., is a table – and asof matches against each record.

In [123]:
t asof ([] sym:`msft`ibm; ti:10:01:01 10:01:03)

qty px 
-------
100 45 
200 160


An as-of join will determine the state “as of” any point in time.

#### 9.9.9 Window Join

Window joins are a generalization of as-of joins and are specifically geared for analyzing the relationship between trades and quotes in finance. The idea is that you want to investigate the behavior of quotes in a neighborhood of each trade. For example, to determine how well a trade was executed, you need to examine the range of bid and ask prices that were prevalent around the trade time.

In [127]:
show t:([]sym:3#`aapl;time:09:30:01 09:30:04 09:30:08;price:100 103 101)


sym  time     price
-------------------
aapl 09:30:01 100  
aapl 09:30:04 103  
aapl 09:30:08 101  


In [126]:
show q:([] sym:8#`aapl;
    time:09:30:01+(til 5),7 8 9;
    ask:101 103 103 104 104 103 102 100;
    bid:98 99 102 103 103 100 100 99)

sym  time     ask bid
---------------------
aapl 09:30:01 101 98 
aapl 09:30:02 103 99 
aapl 09:30:03 103 102
aapl 09:30:04 104 103
aapl 09:30:05 104 103
aapl 09:30:08 103 100
aapl 09:30:09 102 100
aapl 09:30:10 100 99 


In [133]:
w:-2 1+\:t `time / each-left for each (-2, 1)
w

09:29:59 09:30:02 09:30:06
09:30:02 09:30:05 09:30:09


To see all the values in each window, pass the identity function :: in place of the aggregates. The result is similar to grouping without aggregate in a query and is helpful to see what is happening within each window.

In [137]:
wj[w;`sym`time;t;(q;(::;`ask);(::;`bid))]
wj[w;`sym`time;t;(q;(max;`ask);(min;`bid))]

sym  time     price ask             bid           
--------------------------------------------------
aapl 09:30:01 100   101 103         98 99         
aapl 09:30:04 103   103 103 104 104 99 102 103 103
aapl 09:30:08 101   104 103 102     103 100 100   


sym  time     price ask bid
---------------------------
aapl 09:30:01 100   103 98 
aapl 09:30:04 103   104 99 
aapl 09:30:08 101   104 100


**wj** includes quotes that arrive within window [] + quote which was prevailing on entry to the window

**wj1** includes quotes that arrive only within window []

In [138]:
wj1[w;`sym`time;t;(q;(::;`ask);(::;`bid))]

sym  time     price ask             bid           
--------------------------------------------------
aapl 09:30:01 100   101 103         98 99         
aapl 09:30:04 103   103 103 104 104 99 102 103 103
aapl 09:30:08 101   103 102         100 100       


### 9.10 Parameterized Queries

Parameterized queries with templates have restrictions:

- A column cannot be passed as a parameter since columns are part of the syntactic sugar of the template. They only become symbols under the covers.

- Multiple parameters cannot be implicit – i.e., they must be declared explicitly. Although this seems to work for a single parameter, we recommend making all parameters explicit.

In [139]:
t:([] c1:`a`b`c; c2:10 20 30; c3:1.1 2.2 3.3)

In [140]:
select from t where c2>15

c1 c2 c3 
---------
b  20 2.2
c  30 3.3


In [142]:
proc:{[sc] select from t where c2>sc}
proc 15


c1 c2 c3 
---------
b  20 2.2
c  30 3.3


In [144]:
proc2:{[nms;sc] select from t where c1 in nms, c2>sc}
proc2[`a`c; 15]

c1 c2 c3 
---------
c  30 3.3


- You can pass a table as a parameter to a template, either by value of by name.

You can effectively parameterize column names in two ways, only one of which is good practice. 
- First, you can mimic a common technique from SQL in which the **query is created dynamically**: build the query text in a string and then invoke the interpreter programmatically using **value**. This is comparatively slow. Worse, it exposes your application to injection attacks, since any q expression that appears inside the text will be executed.

- The preferred method is to **use the functional form for queries** – see §9.12 – which is fast and secure. In functional form, all columns are referred to by symbolic name, so column names can be passed as symbols. In fact, any component of a query can be passed as an argument in functional form.

### 9.11 Views

A q-sql view is a named table expression created as an alias with the double-colon operator **::**. It is common to use the templates in views but this is not a limitation.

In the following example, contrast u, which is a q variable that is assigned the result of a query, to the view (alias) v that is the query expression itself.

In [146]:
t:([] c1:`a`b`c; c2:10 20 30)
u:select from t where c2>15
v::select from t where c2>15
u
v

c1 c2
-----
b  20
c  30


c1 c2
-----
b  20
c  30


In [148]:
update c2:15 from `t where c1=`b
t
u   / unchanged
v   / changed as underlying table data has changed


`t


c1 c2
-----
a  10
b  15
c  30


c1 c2
-----
b  20
c  30


c1 c2
-----
c  30


To find the underlying query of a view, or any alias, apply the function **view** to the symbol alias name.

In [149]:
view `v

"select from t where c2>15"


In [150]:
a:42
b::a
view `b

,"a"


In [152]:
views `. / all views in namespace

`b`v`view


### 9.12 Functional Forms

The function **parse** can be applied to a string containing a template query to produce a parse tree whose items (nearly) work in the equivalent functional form. A complication is that the operators are displayed in k form instead of q.

The functional forms of the four templates select, exec, update and delete are powerful because they allow all constituents to be parameterized. They can be used for any query but are especially handy for queries that are generated or completed programmatically. The q interpreter parses the syntactic sugar of the templates into their equivalent functional forms, so there is no performance difference.

**We recommend writing table operations in fundamental or template form unless you anticipate needing to parameterize column names.**

There are two functional forms, one for select and exec, the other for update and delete. The types of the arguments passed determine the overload. The forms are,

?[t;c;b;a] / select and exec

![t;c;b;a] / update and delete

where

- t is a table or the name of a table
- a is a dictionary of aggregates
- b is a dictionary of groupbys or a flag controlling other aspects of the query
- c is a list of constraints.
The expressions in a, b and c can involve columns of t and also any variables that are in scope. The rules for expression interpretation are:


- Columns are always represented by their symbolic names
- Consequently, any literal symbols, or lists of symbols, appearing in the expressions must be distinguished. This is done by enlisting them. Really.

In [31]:
t:([] c1:`a`b`a`c`a`b`c; c2:10*1+til 7; c3:1.1*1+til 7)
ft:{([] c1:`a`b`a`c`a`b`c; c2:10*1+til 7; c3:1.1*1+til 7)} /function returning table

#### 9.12.1 Functional select

In [5]:
?[t; (); 0b; ()]~select from t
?[`t; (); 0b; ()]       / table van be passed by name
?[ft[]; (); 0b; ()]     / or parameterized as a result of any q expr

1b


c1 c2 c3 
---------
a  10 1.1
b  20 2.2
a  30 3.3
c  40 4.4
a  50 5.5
b  60 6.6
c  70 7.7


c1 c2 c3 
---------
a  10 1.1
b  20 2.2
a  30 3.3
c  40 4.4
a  50 5.5
b  60 6.6
c  70 7.7


In [4]:
ft
ft[]

{([] c1:`a`b`a`c`a`b`c; c2:10*1+til 7; c3:1.1*1+til 7)}


c1 c2 c3 
---------
a  10 1.1
b  20 2.2
a  30 3.3
c  40 4.4
a  50 5.5
b  60 6.6
c  70 7.7


**Constraints**

The constraint parameter must be a list of q parse trees, one item for each subphrase in the where phrase. The parse tree for each subphrase can be obtained by converting the expression fully to prefix form and then transforming each function into a list with it followed by all its arguments. 
**Remember, columns names become symbols and literal symbols get enlisted.**

In [6]:
select from t where c2>35,c1 in `b`c

c1 c2 c3 
---------
c  40 4.4
b  60 6.6
c  70 7.7


In [7]:
?[t; ((>;`c2;35); (in;`c1;enlist `b`c)); 0b; ()]

c1 c2 c3 
---------
c  40 4.4
b  60 6.6
c  70 7.7


**Aggregation without grouping**

The aggregate parameter is a dictionary whose keys are column names and whose value list is a list of parse trees, one for each subphrase of the aggregate phrase. Again, column names are symbols, and literal symbols get enlisted.

Tip

While q will assign default columns names in the templates, you must explicitly provide all columns names in functional form.

In [8]:
select max c2, c2 wavg c3 from t

c2 c3 
------
70 5.5


In [9]:
?[t; (); 0b; `maxc2`wtavg!((max;`c2); (wavg;`c2;`c3))]

maxc2 wtavg
-----------
70    5.5  


In [32]:
?[t; (); 0b; `c2`c3!(`c2,`c3)]

c2 c3 
------
10 1.1
20 2.2
30 3.3
40 4.4
50 5.5
60 6.6
70 7.7


**Grouping without Aggregation**

As with the aggregate parameter, we construct a dictionary with column names and parse trees of the by subphrases. 
In this particular example, we demonstrate how to handle the case of grouping on a single column. Recall that a singleton dictionary requires the key and value to be enlisted – this enlist is separate from enlisting literal symbols in functional form.

In [11]:
select by c1 from t  / equivalent of last

c1| c2 c3 
--| ------
a | 50 5.5
b | 60 6.6
c | 70 7.7


In [12]:
?[t; (); (enlist `c1)!enlist `c1; ()]

c1| c2 c3 
--| ------
a | 50 5.5
b | 60 6.6
c | 70 7.7


**Constraints + Grouping + Aggregation**

We find code to be more readable if you separately construct the three parameters as variables and pass these in to the functional form.

In [13]:
select max c2, c2 wavg c3 by c1 from t where c2>35,c1 in `b`c

c1| c2 c3 
--| ------
b | 60 6.6
c | 70 6.5


In [14]:
c:((>;`c2;35); (in;`c1;enlist `b`c))
b:(enlist `c1)!enlist `c1
a:`maxc2`wtavg!((max;`c2); (wavg;`c2;`c3))
?[t;c;b;a]

**Distinct**

In this case set the by parameter to 1b

In [15]:
t:([] c1:`a`b`a`c`b`c; c2:1 1 1 2 2 2; c3:10 20 30 40 50 60)
select distinct c1,c2 from t

c1 c2
-----
a  1 
b  1 
c  2 
b  2 


In [16]:
?[t; (); 1b; `c1`c2!`c1`c2]

c1 c2
-----
a  1 
b  1 
c  2 
b  2 


**select[n]**

In [ ]:
The extended form select[n] adds a fifth parameter to the functional form

In [18]:
t:([] c1:`a`b`c; c2:10 20 30)

In [20]:
select[2] from t
?[t;();0b;();2]

c1 c2
-----
a  10
b  20


c1 c2
-----
a  10
b  20


**select[>ci]**

The extended form select[>ci] adds two additional parameters to the functional form. The first is the initial value for the comparison and the second is a list with the k form – seriously! – of the comparison operator for the sort along with the column name for the sort.

In [22]:
select[>c1] c1,c2 from t

c1 c2
-----
a  10
b  20
c  30


In [23]:
?[t;();0b;`c1`c2!`c1`c2; 0W; (>:;`c1)]

c1 c2
-----
c  30
b  20
a  10


#### 9.12.2 Functional exec

The functional form for exec on a single result column depends on whether you want a list or dictionary to be returned. 
- Assuming there is no grouping, use the empty list for the by parameter. 
- For a list result, specify the aggregate as a parse tree; 
- for a dictionary result, specify a dictionary mapping the result name to its parse tree.

In [24]:
t:([] c1:`a`b`c`a; c2:10 20 30 40; c3:1.1 2.2 3.3 4.4)

In [25]:
/ no grouping, list returned
exec distinct c1 from t
?[t; (); (); (distinct; `c1)]

`a`b`c


`a`b`c


In [27]:
/ no grouping, single column, dictionary returned
exec c1:distinct c1 from t
?[t; (); (); (enlist `c1)!enlist (distinct; `c1)]

c1| a b c


c1| a b c


In [34]:
/ no grouping, multiple columns, dictionary returned
exec distinct c1, c2 from t
?[t; (); (); `c1`c2!((distinct; `c1); `c2)]

c1| `a`b`c
c2| 10 20 30 40 50 60 70


c1| `a`b`c
c2| 10 20 30 40 50 60 70


In [35]:
/To group on a single column, specify its symbol name in the by parameter.
exec c2 by c1 from t
?[t; (); `c1; `c2]

a| 10 30 50
b| 20 60
c| 40 70


a| 10 30 50
b| 20 60
c| 40 70


#### 9.12.3 Functional update

The syntax of functional form of update is identical to that of select except that ! is used in place of ?. 

In the following examples you will need to keep track of the **different uses of enlist:**

- making a list of parse trees from a single parse expression
- creating singleton dictionaries
- distinguishing literal symbols from column names.

In [41]:
t:([] c1:`a`b`c`a`b; c2:10 20 30 40 50)
t

c1 c2
-----
a  10
b  20
c  30
a  40
b  50


In [37]:
update c2:100 from t where c1=`a

c1 c2 
------
a  100
b  20 
c  30 
a  100
b  50 


In [38]:
c:enlist (=;`c1;enlist `a)
b:0b
a:(enlist `c2)!enlist 100

In [39]:
![t;c;b;a]

c1 c2 
------
a  100
b  20 
c  30 
a  100
b  50 


In [40]:
update c2:sums c2 by c1 from t

c1 c2
-----
a  10
b  20
c  30
a  50
b  70


In [43]:
![`t; (); (enlist `c1)!enlist `c1; (enlist `c2)!enlist(sums; `c2)]

`t


#### 9.12.4 Functional delete

The syntax of functional delete is a simplified form of functional update.

![t;c;0b;a]

where 
- t is a table, or the name of a table, 
- c is a list of parse trees for where subphrases and 
- a is a list of column names. 

Either c or a, but not both, must be present. 
- If c is present, it specifies which rows are to be deleted. In this case you must specify a as an empty list of symbols.
- If a is present it is a list of symbol column names to be deleted. 

In [46]:
t:([] c1:`a`b`c`a`b; c2:10 20 30 40 50)
t

c1 c2
-----
a  10
b  20
c  30
a  40
b  50


In [49]:
delete from t where c1=`b
![t;enlist (=;`c1;enlist `b);0b;`symbol$()]

c1
--
a 
c 
a 


c1
--
a 
c 
a 


[0;31mtype[0m: [0;31mtype[0m

In [49]:
delete c2 from t
![`t;();0b;enlist `c2]
t

c1
--
a 
b 
c 
a 
b 


`t


c1
--
a 
b 
c 
a 
b 


### 9.13 Examples

In [5]:
mktrades:{[tickers; sz]
  dt:2015.01.01+sz?31;
  tm:sz?24:00:00.000;
  sym:sz?tickers;
  qty:10*1+sz?1000;
  px:90.0+(sz?2001)%100;
  t:([] dt; tm; sym; qty; px);
  t:`dt`tm xasc t;
  t:update px:6*px from t where sym=`goog;
  t:update px:2*px from t where sym=`ibm;
  t}
trades:mktrades[`aapl`goog`ibm; 10000000]

In [11]:
trades

dt         tm           sym  qty  px    
----------------------------------------
2015.01.01 00:00:00.141 ibm  1000 192.34
2015.01.01 00:00:00.425 goog 390  599.4 
2015.01.01 00:00:00.902 goog 9240 620.52
2015.01.01 00:00:01.425 aapl 9770 91.21 
2015.01.01 00:00:02.868 aapl 6620 107.63
2015.01.01 00:00:02.870 ibm  2240 218.12
2015.01.01 00:00:03.286 aapl 3920 95.07 
2015.01.01 00:00:03.638 goog 9610 549.9 
2015.01.01 00:00:03.671 ibm  4350 188.1 
2015.01.01 00:00:04.199 aapl 9390 109.32
2015.01.01 00:00:04.586 ibm  4870 198.18
2015.01.01 00:00:04.723 ibm  5180 195.26
2015.01.01 00:00:05.586 goog 1100 593.34
2015.01.01 00:00:05.715 ibm  9820 212.2 
2015.01.01 00:00:06.098 aapl 8120 94.78 
2015.01.01 00:00:06.249 goog 7600 560.88
2015.01.01 00:00:06.261 ibm  7390 205.32
2015.01.01 00:00:06.464 ibm  9860 181.24
2015.01.01 00:00:06.753 aapl 710  94.97 
2015.01.01 00:00:07.017 aapl 2020 107.07
..


In [3]:
instr:([sym:`symbol$()] name:`symbol$(); industry:`symbol$())
`instr upsert (`ibm; `$"International Business Machines"; `$"Computer Services")
`instr upsert (`msft; `$"Microsoft"; `$"Software")
`instr upsert (`goog; `$"Google"; `$"Search")
`instr upsert (`aapl; `$"Apple"; `$"Electronics")
instr

`instr


`instr


`instr


`instr


sym | name                            industry         
----| -------------------------------------------------
ibm | International Business Machines Computer Services
msft| Microsoft                       Software         
goog| Google                          Search           
aapl| Apple                           Electronics      


In [6]:
meta trades
update `instr$sym from `trades  / making instr FK
meta trades

c  | t f a
---| -----
dt | d   s
tm | t    
sym| s    
qty| j    
px | f    


`trades


c  | t f     a
---| ---------
dt | d       s
tm | t        
sym| s instr  
qty| j        
px | f        


We find the high, low and close over one-minute intervals for Google.

In [7]:
select hi:max px,lo:min px,open:first px, close:last px by dt,tm.minute from trades where sym=`goog

dt         minute| hi     lo     open   close 
-----------------| ---------------------------
2015.01.01 00:00 | 658.74 542.46 599.4  657.96
2015.01.01 00:01 | 658.2  542.1  627.66 617.16
2015.01.01 00:02 | 657.54 540.36 638.64 608.22
2015.01.01 00:03 | 659.76 540.24 549.06 598.8 
2015.01.01 00:04 | 658.92 543.9  653.94 594.54
2015.01.01 00:05 | 656.88 541.32 654.9  612.3 
2015.01.01 00:06 | 650.46 540.48 601.44 611.22
2015.01.01 00:07 | 659.22 540    622.08 619.62
2015.01.01 00:08 | 657.06 544.02 566.76 643.68
2015.01.01 00:09 | 659.7  544.38 617.94 659.7 
2015.01.01 00:10 | 657.72 545.64 608.64 583.02
2015.01.01 00:11 | 655.86 541.32 618.48 557.4 
2015.01.01 00:12 | 654.42 541.44 615.54 628.86
2015.01.01 00:13 | 658.02 542.82 657.24 604.2 
2015.01.01 00:14 | 658.62 540.84 551.76 578.34
2015.01.01 00:15 | 659.46 540.42 652.26 593.1 
2015.01.01 00:16 | 658.92 540    551.22 555.42
2015.01.01 00:17 | 659.04 540.18 581.7  634.86
2015.01.01 00:18 | 659.76 541.14 615.72 642.6 
2015.01.01 00

**Daily VWAP**

In [8]:
select vwap:qty wavg px by dt from trades where sym=`ibm

dt        | vwap    
----------| --------
2015.01.01| 199.9572
2015.01.02| 200.0544
2015.01.03| 199.9884
2015.01.04| 199.9906
2015.01.05| 199.9701
2015.01.06| 200.0659
2015.01.07| 199.9902
2015.01.08| 199.967 
2015.01.09| 199.9984
2015.01.10| 199.9598
2015.01.11| 199.9733
2015.01.12| 199.9704
2015.01.13| 199.9974
2015.01.14| 200.0424
2015.01.15| 200.0508
2015.01.16| 200.0404
2015.01.17| 199.9596
2015.01.18| 199.9784
2015.01.19| 200.0056
2015.01.20| 199.993 
..


**100ms VWAP**

In [9]:
select vwap:qty wavg px by dt,100 xbar tm from trades where sym=`ibm

dt         tm          | vwap    
-----------------------| --------
2015.01.01 00:00:00.100| 192.34  
2015.01.01 00:00:02.800| 218.12  
2015.01.01 00:00:03.600| 188.1   
2015.01.01 00:00:04.500| 198.18  
2015.01.01 00:00:04.700| 195.26  
2015.01.01 00:00:05.700| 212.2   
2015.01.01 00:00:06.200| 205.32  
2015.01.01 00:00:06.400| 181.24  
2015.01.01 00:00:09.400| 213.34  
2015.01.01 00:00:09.800| 198.2643
2015.01.01 00:00:09.900| 210.54  
2015.01.01 00:00:10.100| 184.98  
2015.01.01 00:00:10.500| 190.82  
2015.01.01 00:00:13.300| 192.58  
2015.01.01 00:00:13.500| 182.7   
2015.01.01 00:00:13.700| 209.28  
2015.01.01 00:00:15.300| 212.5   
2015.01.01 00:00:15.500| 194.42  
2015.01.01 00:00:16.000| 197.04  
2015.01.01 00:00:17.600| 185.4   
..


We use fby to select records attaining the maximum price each day for each sym.

In [12]:
select from trades where px=(max;px) fby sym

px 
---
110
220
660


#### operations on nested columns

**Denormalizing**

- grouping without aggregation using select
- using xgroup

The difference is that xgroup does not automatically sort on the result key column.

In [15]:
dntrades:select dt,tm,qty,px by sym from trades
dntrades~`sym xasc `sym xgroup trades

1b


In [16]:
dntrades

sym | dt                                                                     ..
----| -----------------------------------------------------------------------..
aapl| 2015.01.01 2015.01.01 2015.01.01 2015.01.01 2015.01.01 2015.01.01 2015...
goog| 2015.01.01 2015.01.01 2015.01.01 2015.01.01 2015.01.01 2015.01.01 2015...
ibm | 2015.01.01 2015.01.01 2015.01.01 2015.01.01 2015.01.01 2015.01.01 2015...


In [20]:
select 2#dt,2#tm,2#qty,2#px by sym from trades / to make structure more evident

sym | dt                    tm                        qty       px           
----| -----------------------------------------------------------------------
aapl| 2015.01.01 2015.01.01 00:00:01.425 00:00:02.868 9770 6620 91.21  107.63
goog| 2015.01.01 2015.01.01 00:00:00.425 00:00:00.902 390  9240 599.4  620.52
ibm | 2015.01.01 2015.01.01 00:00:00.141 00:00:02.870 1000 2240 192.34 218.12


Structure of denormalised table is key column and multiple columns which are  nested lists. In such a table with compound columns – i.e., lists of simple lists of the same type – you will need adverbs for column operations. Lots of adverbs.

In [21]:
select sym,cnt:count each dt,avgpx:avg each px from dntrades

sym  cnt     avgpx   
---------------------
aapl 3333109 100.0047
goog 3334279 600.0015
ibm  3332612 200.0041


To find the volume-weighted average price by we use the each-both adverb ' with the dyadic wavg.

In [22]:
select sym,vwap:qty wavg' px from dntrades

sym  vwap    
-------------
aapl 100.0082
goog 599.996 
ibm  200.0074


**Profit of the ideal transaction over the month for each symbol.** This is the maximum amount of money that could be made with complete foreknowledge of the market. In other words, find the largest profit obtainable by buying at some traded price and selling at the highest subsequently traded price. To solve this, we reverse the perspective. For each traded price, we look at the minimum of prices that preceded it. The largest such difference is our answer. The maximum draw down is dual.

In [23]:
select max px-mins px by sym from trades

sym | px 
----| ---
aapl| 20 
goog| 120
ibm | 40 


In [24]:
select min px-maxs px by sym from trades

sym | px  
----| ----
aapl| -20 
goog| -120
ibm | -40 


### 9.14. Pivot table

In [1]:
show t:([]k:1 2 3 2 3; p:`a1`a2`a1`a3`a2; v:100 200 300 400 500)


k p  v  
--------
1 a1 100
2 a2 200
3 a1 300
2 a3 400
3 a2 500


1. Collect the unique values of the pivot column p into a list P.

In [2]:
P:exec distinct p from t

2. Write a query that extracts the key-value pairs for p and v grouped by k.

In [4]:
t[`p],'t[`v]

`a1 100
`a2 200
`a1 300
`a3 400
`a2 500


In [7]:
select (p,'v) by k from t / table with list of pairs
select p!v by k from t    / table with dictionaries

k| v                    
-| ---------------------
1| ,(`a1;100)           
2| ((`a2;200);(`a3;400))
3| ((`a1;300);(`a2;500))


k| v             
-| --------------
1| (,`a1)!,100   
2| `a2`a3!200 400
3| `a1`a2!300 500


In [6]:
exec p!v by k from t / dictionary of dictionaries

1| (,`a1)!,100
2| `a2`a3!200 400
3| `a1`a2!300 500


3. Enhance the previous query to produce a keyed table by rectangularizing the dictionaries by filling missing values using P#. Magic happens. Observe that we need to name the resulting key column explicitly.

In [25]:
P#`a1`a3!1 3  / P is the list of unique values in column P; adds missing values 

a1| 1
a2| 
a3| 3


In [46]:
exec P#p!v by k:k from t

k| a1  a2  a3 
-| -----------
1| 100        
2|     200 400
3| 300 500    


4. Write the query to extract the unique values of the pivot column in functional form.

In [26]:
P:?[t; (); (); (distinct; `p)]

5. Convert the pivot query to functional form.

In [27]:
?[t;(); (1#`k)!1#`k; (#;`P;(!;`p;`v))]

k| a1  a2  a3 
-| -----------
1| 100        
2|     200 400
3| 300 500    


6. Place the previous functional forms in a function that takes the table and the column names as parameters and returns the pivot result.

In [28]:
dopivot:{[t; kn; pn; vn]
  P:?[t; (); (); (distinct; pn)];
  ?[t;(); (1#kn)!1#kn; (#;`P;(!;pn;vn))]}
dopivot[t;`k;`p;`v]

k| a1  a2  a3 
-| -----------
1| 100        
2|     200 400
3| 300 500    


7. Write an expression that converts the fields of the pivot column to valid names when they are not – e.g., integers. In general, producing unique, valid names is a non-trivial exercise. Find a solution that (at least) works on tn below.

In [30]:
show tn:([] k:1 2 3 2 3; p:(`a1;2;`a1;3;2); v:100 200 300 400 500)


k p   v  
---------
1 `a1 100
2 2   200
3 `a1 300
2 3   400
3 2   500


In [31]:
mkNames:{
  x:(::),x;       / added dummy element so it becomes list?
  x:1_x:@[x; where not 10h=type each x; string];   / converted each value to string
  `$@[x; where not any x[;0] within/: ("AZ";"az"); "X_",]} / if first char is not letter add X_


In [32]:
dopivot:{[t; kn; pn; vn]
  t:![t; (); 0b; (1#pn)!enlist (`mkNames; pn)];  / apply mkNames to all  values in column
  P:?[t; (); (); (distinct; pn)];
  ?[t;(); (1#kn)!1#kn; (#;`P;(!;pn;vn))]}


In [33]:
dopivot[tn;`k;`p;`v]

k| a1  X_2 X_3
-| -----------
1| 100        
2|     200 400
3| 300 500    


In [36]:
x:(`a1;3;`a7)
x:(::),x
x

::
`a1
3
`a7


In [38]:
where not 10h=type each x

0 1 2 3


In [41]:
1_x:@[x; where not 10h=type each x; string]

"a1"
,"3"
"a7"


8. Next, we generalize to the case when there are multiple entries in v for a single key value. For example, applied to tr below, our current dopivot misses the 1000 value in v.

In [43]:
/ 2 values for (1;a1): 100 and 1000
show tr:([]k:1 2 3 2 3 1; p:`a1`a2`a1`a3`a2`a1; v:100 200 300 400 500 1000)

k p  v   
---------
1 a1 100 
2 a2 200 
3 a1 300 
2 a3 400 
3 a2 500 
1 a1 1000


We would like to apply an aggregate function such as sum to obtain,

In [47]:
dopivot:{[t; agg; kn; pn; vn]
  t:![t; (); 0b; (1#pn)!enlist (`mkNames; pn)];  / apply mkNames to all  alues in column
  t:?[t; (); (kn,pn)!kn,pn; (1#vn)!enlist (agg;vn)];
  P:?[t; (); (); (distinct; pn)];
  ?[t;(); (1#kn)!1#kn; (#;`P;(!;pn;vn))]}

In [48]:
dopivot[tr;sum;`k;`p;`v]

k| a1   a2  a3 
-| ------------
1| 1100        
2|      200 400
3| 300  500    


9. We would like to handle the case of compound keys – e.g., k1 and k2 in tk below.

In [49]:
k1:1 2 3 2 3 1 1
k2:10 20 30 40 50 60 10
p:`a1`a2`a1`a3`a2`a1`a1
v:100 200 300 400 500 1000 10000
show tk:([]k1:k1; k2:k2; p:p; v:v)

k1 k2 p  v    
--------------
1  10 a1 100  
2  20 a2 200  
3  30 a1 300  
2  40 a3 400  
3  50 a2 500  
1  60 a1 1000 
1  10 a1 10000


Modify dopivot to take a list of key column names and test it on tk.

In [50]:
dopivot:{[t; agg; ks; pn; vn]
  ks,:();
  t:![t; (); 0b; (1#pn)!enlist (`mkNames; pn)];
  t:?[t; (); (ks,pn)!ks,pn; (1#vn)!enlist (agg;vn)];
  P:?[t; (); (); (distinct; pn)];
  ?[t;(); ks!ks; (#;`P;(!;pn;vn))]}
dopivot[tk;sum;`k1`k2;`p;`v]

k1 k2| a1    a2  a3 
-----| -------------
1  10| 10100        
1  60| 1000         
2  20|       200    
2  40|           400
3  30| 300          
3  50|       500    


10. Modify dopivot to accept a string argument representing a valid where phrase for the input table and use it to constrain the pivot. Test it against tk with the phrase k1<>2.

In [51]:
dopivot:{[t; agg; wh; ks; pn; vn]
  ks,:();
  c:enlist parse wh;
  t:?[t; c; 0b; (cols t)!cols t];
  t:![t; (); 0b; (1#pn)!enlist (`mkNames; pn)];
  t:?[t; (); (ks,pn)!ks,pn; (1#vn)!enlist (agg;vn)];
  P:?[t; (); (); (distinct; pn)];
  ?[t; (); ks!ks; (#;`P;(!;pn;vn))]}
dopivot[tk;sum;"k1<>2";`k1`k2;`p;`v]

k1 k2| a1    a2 
-----| ---------
1  10| 10100    
1  60| 1000     
3  30| 300      
3  50|       500


11. Finally, modify dopivot to accept an empty list in the aggregate or the where parameters. In the former case, use first as the default aggregate; in the latter, perform no constraint. Test against tk with empty list arguments for both.

In [52]:
dopivot:{[t; agg; wh; ks; pn; vn]
  ks,:();
  agg:first agg,first;
  c:$[count wh; enlist parse wh; ()];
  t:?[t; c; 0b; (cols t)!cols t];
  t:![t; (); 0b; (1#pn)!enlist (`mkNames; pn)];
  t:?[t; (); (ks,pn)!ks,pn; (1#vn)!enlist (agg;vn)];
  P:?[t; (); (); (distinct; pn)];
  ?[t; (); ks!ks; (#;`P;(!;pn;vn))]}
dopivot[tk;();"";`k1`k2;`p;`v]

k1 k2| a1   a2  a3 
-----| ------------
1  10| 100         
1  60| 1000        
2  20|      200    
2  40|          400
3  30| 300         
3  50|      500    


In [53]:
t:`date xasc ([] date:2015.01.01+1000000?10; sym:1000000?`aapl`ibm`intc; qty:1+1000000?100)
dopivot[t; sum; ""; `date; `sym; `qty] / ymmv

date      | aapl    ibm     intc   
----------| -----------------------
2015.01.01| 1693065 1668499 1700074
2015.01.02| 1667313 1686058 1680353
2015.01.03| 1679478 1682674 1689045
2015.01.04| 1703002 1682709 1668598
2015.01.05| 1683069 1688756 1693817
2015.01.06| 1678786 1674017 1679108
2015.01.07| 1685413 1690676 1680870
2015.01.08| 1675548 1692420 1688387
2015.01.09| 1680614 1663857 1667443
2015.01.10| 1680781 1690581 1678134
